In [1]:
import os
from utils import *
import optuna

In [2]:
data, usermap, itemmap, users = load_data2()
data_train, data_val = split_data2(data, 0.2)

In [3]:
study_name = "NFM"
study = optuna.create_study(
    study_name=study_name,
    storage=get_database_url(),
    load_if_exists=True,
    direction="maximize",
)

[I 2023-12-20 16:45:33,486] A new study created in RDB with name: NFM


In [5]:
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

def objective(trial):
    num_factors = trial.suggest_int('num_factors', 1, 1000)
    l1_ratio = trial.suggest_float('l1_ratio', 0, 1)
    solver = trial.suggest_categorical('solver', ['coordinate_descent', 'multiplicative_update'])
    init_type = trial.suggest_categorical('init_type', ['random', 'nndsvda'])
    beta_loss = trial.suggest_categorical('beta_loss', ['frobenius', 'kullback-leibler'])
    
    recommender = NMFRecommender(data_train, verbose=False)
    recommender.fit(
        num_factors=num_factors,
        l1_ratio=l1_ratio,
        solver=solver,
        init_type=init_type,
        beta_loss=beta_loss,
        verbose=False
    )
    _, _, ev_map, _, _ = evaluator(recommender, data_train, data_val)
    
    return ev_map

study.optimize(objective, n_trials=150)

In [ ]:
study.best_params

In [ ]:
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender

recommender = NMFRecommender(data_train, verbose=False)
recommender.fit(**study.best_params)

In [ ]:
pr, re, ev_map, ev_ndcg, ev_auc = evaluator(recommender, data_train, data_val)
print("Precision: {:.8f}".format(pr))
print("Recall: {:.8f}".format(re))
print("MAP: {:.8f}".format(ev_map))

In [ ]:
submission2(recommender, users, usermap, itemmap, data_train)